# Watershed Segmentation

In [ ]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.color import rgb2gray
 
def watershed_segmentation_dist(labels,display=False):
    
    #performs watershed segmentation where the distance function is the distance to the edge of the label
    
    #labels: an array of numbers, 0 value for background and value i for the i^th label
    #display: gives a picture of our outputs
    
    #returns: new labels for the image after being watershed segmented
    
    distance = ndi.distance_transform_edt(labels)
    local_maxi = peak_local_max(distance, indices=False, labels=labels)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=labels)
    if display == True:
        fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True,
                                 subplot_kw={'adjustable': 'box-forced'})
        ax = axes.ravel()
        ax[0].imshow(labels, cmap=plt.cm.gray, interpolation='nearest')
        ax[0].set_title('Overlapping objects')
        ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
        ax[1].set_title('Distances')
        ax[2].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest')
        ax[2].set_title('Separated objects')
        for a in ax:
            a.set_axis_off()
        fig.tight_layout()
        plt.show()
    return labels

def watershed_segmentation_dark(image,labels,display=False):
    
    #performs watershed segmentation where the distance function is the lightness of the grayscale image
    
    #image: the image we have labels of (assumed to be inverted if cells are black and background white)
    #labels: an array of numbers, 0 value for background and value i for the i^th label
    #display: gives a picture of our outputs
    
    #returns: new labels for the image after being watershed segmented
    
    im_gray = rgb2gray(image)
    local_maxi = peak_local_max(im_gray, indices=False, labels=labels)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=labels)
    if display == True:
        fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True,
                                 subplot_kw={'adjustable': 'box-forced'})
        ax = axes.ravel()
        ax[0].imshow(labels, cmap=plt.cm.gray, interpolation='nearest')
        ax[0].set_title('Overlapping objects')
        ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
        ax[1].set_title('Distances')
        ax[2].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest')
        ax[2].set_title('Separated objects')
        for a in ax:
            a.set_axis_off()
        fig.tight_layout()
        plt.show()
    return labels

# Histogram of Orientated Gradients

In [ ]:
import cv2
from skimage.color import rgb2gray
import numpy as np

def HoG(img):
    
    #calculates the histogram of orientated gradients
    
    #image: the image we wish to calculated on
    
    #returns: the magnitude of the x and y gradients
    
    img = rgb2gray(img)
    img = np.float32(img) / 255.0
 
    # Calculate gradient 
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)

    # Python Calculate gradient magnitude and direction (in degrees) 
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    
    return mag

# k-means Clustering with Colours

In [ ]:
from sklearn.cluster import KMeans

def kmeanscluster(image,k,merge=False):
    
    #Calculates the k means clustering of the colours in the image
    
    #img: the four channel image we wish to work with
    #k: the k in k-means clustering
    #merge: an optional argument which gives a merging of non-background clusters to get a mask
    
    #returns either the mask if we are merging, or the labels if we are not
    
    image = image[:,:,:3]
    image_array = image.reshape((image.shape[0] * image.shape[1], 3))
    # Clusters the pixels
    clt = KMeans(n_clusters = k)
    clt.fit(image_array)
    labels = clt.labels_
    if merge==True:
        counts = np.bincount(labels)
        labels = labels.reshape(image.shape[0],image.shape[1])
        background = np.argmax(counts)
        mask = np.where(labels != background, 1, 0)
        return mask
    else:
        return labels

# Image Channel Scaling

In [ ]:
#Reference: Johan Steunenberg

IMG_CHANNELS = 3

def scale_img_channels(img):
    
    #Scales the image channels linearly to use the entire range of 0 to 255
    
    #img: the 4 channel image we are rescaling
    
    #returns: the channel rescaled image
    
    for i in range(IMG_CHANNELS):
        channel = img[:,:,i]
        channel = channel - channel.min()
        channelmax = channel.max()
        if channelmax > 0:
            factor = 255/channelmax
            channel = (channel * factor).astype(int)
        img[:,:,i] = channel
    return img